<a href="https://colab.research.google.com/github/TamNguyenMinh2494/Landscape-Image-Labelling-Using-Object-Detection/blob/master/Inference_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive



#Using Frozen model

Mounting GDrive and copy the model

In [0]:
!cp "drive/My Drive/Google_landscapist/Object_Detection/model-201912241800.zip" ./

In [0]:
!unzip model-201912241800.zip

Archive:  model-201912241800.zip
   creating: inference_graph/
   creating: inference_graph/saved_model/
  inflating: inference_graph/saved_model/saved_model.pb  
   creating: inference_graph/saved_model/variables/
  inflating: inference_graph/pipeline.config  
  inflating: inference_graph/model.ckpt.data-00000-of-00001  
  inflating: inference_graph/frozen_inference_graph.pb  
  inflating: inference_graph/checkpoint  
  inflating: inference_graph/model.ckpt.meta  
  inflating: inference_graph/model.ckpt.index  


# Setup environment

In [0]:
!git clone https://github.com/tensorflow/models.git
!pip install tensorflow-object-detection-api

Cloning into 'models'...
remote: Enumerating objects: 33473, done.
remote: Total 33473 (delta 0), reused 0 (delta 0), pack-reused 33473
Receiving objects: 100% (33473/33473), 511.86 MiB | 4.20 MiB/s, done.
Resolving deltas: 100% (21405/21405), done.
Checking out files: 100% (3194/3194), done.
     |████████████████████████████████| 583kB 42.3MB/s 
     |████████████████████████████████| 61kB 10.4MB/s 
     |████████████████████████████████| 2.3MB 50.6MB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
  Created wheel for tensorflow-object-detection-api: filename=tensorflow_object_detection_api-0.1.1-cp36-none-any.whl size=844514 sha256=7bf78ab8aaf47bb6b127929f0bc5880c66f312ef54d22557926f9ade848fb19c
  Stored in directory: /root/.cache/pip/wheels/4a/54/d0/cfca11930c4b2025d40dede77059094070a67cc3e7bd3b285f
Successfully built tensorflow-object-detection-api


In [0]:
!cd models/research && python setup.py build && python setup.py install

running build
running build_py
creating build
creating build/lib
creating build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph_lib_test.py -> build/lib/object_detection
copying object_detection/model_lib_v2_test.py -> build/lib/object_detection
copying object_detection/model_lib_v2.py -> build/lib/object_detection
copying object_detection/export_tflite_ssd_graph.py -> build/lib/object_detection
copying object_detection/exporter.py -> build/lib/object_detection
copying object_detection/eval_util_test.py -> build/lib/object_detection
copying object_detection/model_lib_test.py -> build/lib/object_detection
copying object_detection/exporter_test.py -> build/lib/object_detection
copying object_detection/model_tpu_main.py -> build/lib/object_detection
copying object_detection/model_main.py -> build/lib/object_detection
copying object_detection/eval_util.py -> build/lib/object_detectio

In [0]:
import sys
sys.path.append("/models/research")

In [0]:
!cd models/research/ && export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

# Imports

In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from distutils.version import StrictVersion
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("models/research")
from object_detection.utils import ops as utils_ops

if StrictVersion(tf.__version__) < StrictVersion('1.12.0'):
  raise ImportError('Please upgrade your TensorFlow installation to v1.12.*.')

# Env Setup

In [0]:
# This is needed to display the images.
%matplotlib inline

In [0]:
!cd models/research && protoc object_detection/protos/*.proto --python_out=.

# Object detection imports
Here are the imports from the object detection module.

In [0]:
from object_detection.utils import label_map_util

from object_detection.utils import visualization_utils as vis_util

# Model preparation 
Make sure these files locate in exactly directory

In [0]:
MODEL_NAME = ''
PATH_TO_FROZEN_GRAPH = MODEL_NAME + '/content/inference_graph/frozen_inference_graph.pb'
PATH_TO_LABELS = "/content/drive/My Drive/Google_landscapist/Object_Detection/label_map.pbtxt"

# Load a (frozen) Tensorflow model into memory.

In [0]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_FROZEN_GRAPH, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

In [0]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

# Helper code

In [0]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [0]:
!wget http://tbck.vn/stores/news_dataimages/haund/112018/30/15/nha-o-nong-thon-thieu-nhung-ban-ve-chuyen-nghiep-15-.6797.jpg
# !wget image's url

In [0]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
#PATH_TO_TEST_IMAGES_DIR = 
TEST_IMAGE_PATHS = ['/content/DGT89.jpg']

# Size, in inches, of the output images.
IMAGE_SIZE = (24, 20)
#ImageFont.truetype('arial.ttf', 30)

In [0]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[1], image.shape[2])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: image})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.int64)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict

In [0]:
import json

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  
  obj = {
      "boxes":output_dict['detection_boxes'].tolist(),
      "classes":output_dict['detection_classes'].tolist(),
      "scores":output_dict['detection_scores'].tolist()
  }
  
  result = open(image_path+'.json','w')
  json.dump(obj,result)
  result.close()
  
  
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=True,
      line_thickness=8)
  
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  
  plt.savefig(image_path+"-result.png")

In [0]:
# output results
def show_results(threshold=0.3):
  objects=['Co','XaBan','Lua','Rau','Bap','Cu','Mia','CayAnTrai','CayXanh','Re','Cafe','CaoSu','Tieu','Tra','Dua','Xe','MatDuong','LeDuong','CongTrinhPhu','CauDuong','Container','TauThuy','CanCau','CauCang','ĐuongRay','TauLua','ĐoChoi','Hoa','HoBoi','CongBenhVien','BacSi','BangCa','CongTruong','HocSinh','PhongHoc','Logo','Toanha','NhaO','HangRao','ChungCu','NhaMay','NhaKho','Thuyen','NhaOChuot','Suoi-Thac','BoHo','Be','Ca','Phao','MaySucKhi','Luoi']
  classlist=output_dict['detection_classes'].tolist()
  scorelist=output_dict['detection_scores'].tolist()
  for i in range(len(classlist)):
    if(scorelist[i]>threshold):
      print(objects[classlist[i]-1],":",round(scorelist[i]*100,0),"%")
show_results()

Container : 39.0 %
Container : 31.0 %


In [0]:
import json

for image_path in TEST_IMAGE_PATHS:
  image = Image.open(image_path)
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = load_image_into_numpy_array(image)
  # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
  image_np_expanded = np.expand_dims(image_np, axis=0)
  # Actual detection.
  output_dict = run_inference_for_single_image(image_np_expanded, detection_graph)
  
  obj = {
      "boxes":output_dict['detection_boxes'].tolist(),
      "classes":output_dict['detection_classes'].tolist(),
      "scores":output_dict['detection_scores'].tolist()
  }
  
  result = open(image_path+'.json','w')
  json.dump(obj,result)
  result.close()
  
  
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      min_score_thresh=.3,
      instance_masks=output_dict.get('detection_masks'),
      use_normalized_coordinates=False,
      line_thickness=8
      )
  
  plt.figure(figsize=IMAGE_SIZE)
  plt.imshow(image_np)
  
  plt.savefig(image_path+"-result.png")